In [1]:
import pandas as pd
import numpy as np
import os

path = 'raw'
os.chdir(path)

df = pd.read_csv('20201103__wv__general__precinct.csv')
df = df.replace(np.nan, '', regex = True)
df = df.drop(['vtd'], axis=1)

In [2]:
# following candidate were listed with wrong party (changed from Mountain to Libertarian)

df.loc[df.candidate == 'ROBERT E. SMITH', 'party'] = 'LBN'
df.loc[df.candidate == 'HOWIE HAWKINS', 'party'] = 'LBN'

In [3]:
df['party_detailed'] = df['party'].replace({'DEM': 'DEMOCRAT',
                                            'REP': 'REPUBLICAN',
                                            'MTN': 'MOUNTAIN',
                                            'CST': 'CONSTITUTION',
                                            'IND': 'INDEPENDENT',
                                            'LBN': 'LIBERTARIAN'})

df = df.drop(['party'], axis=1)

In [20]:
df['party_simplified'] = df['party_detailed'].replace({'MOUNTAIN': 'OTHER',
                                            'INDEPENDENT': 'OTHER',
                                            'CONSTITUTION': 'OTHER'})

In [5]:
def getDistrict(x):
    if 'nd' in x:
        index = x.index('nd')
        return x[:index]
    elif 'th' in x:
        index = x.index('th')
        return x[:index]
    elif 'rd' in x:
        index = x.index('rd')
        return x[:index]
    elif 'st' in x:
        index = x.index('st')
        return x[:index]
    elif 'st' in x:
        index = x.index('st')
        return x[:index]
    else: 
        return x
    
df['district'] = df['district'].apply(getDistrict)
df.district = df.district.str.strip()
df["district"]= df["district"].str.zfill(3)

df.district = df.district.str.replace('000','', regex=False)

In [6]:
df["candidate"] = df["candidate"].str.replace('.', '', regex=True)
df["candidate"] = df["candidate"].str.replace(',', '', regex=True)
df["candidate"] = df["candidate"].str.replace('(', '"', regex=True)
df["candidate"] = df["candidate"].str.replace(')', '"', regex=True)

def cleanCandidate(x):
    if 'REP ' in x:
        index = x.index('REP')
        return x[index+4:]
    elif 'DEM ' in x:
        index = x.index('DEM')
        return x[index+4:]
    elif 'IND ' in x:
        index = x.index('IND')
        return x[index+4:]
    elif 'LBN ' in x:
        index = x.index('LBN')
        return x[index+4:]
    elif 'MTN ' in x:
        index = x.index('MTN')
        return x[index+4:]
    elif 'DuRON JACKSON' in x:
        return 'DURON JACKSON'
    elif "GREG ''HOOTIE '' SMITH" in x:
        return 'GREG "HOOTIE" SMITH'
    elif "NO CANDIDATE(S) NOMINATED" in x:
        return "NO CANDIDATE(S) NOMINATED"
    elif "YES " in x or "For " in x or "Yes" in x or "FOR" == x or "FOR " in x:
        return 'YES'
    elif "NO " in x or "Against " in x or "No" == x or "AGAINST" in x:
        return 'NO'
    elif "Write-in" in x or "WRITE-IN" in x:
        return 'WRITEIN'
    else: 
        return x
    
df['candidate'] = df['candidate'].apply(cleanCandidate)


In [7]:
df.loc[df.office.str.contains('FOR CITY COUNCIL DUNBAR '), 'district'] = df['office']
df.loc[df.office.str.contains('FAIRMONT DISTRICT '), 'district'] = df['office']
df.loc[df.office.str.contains('FOR CITY COUNCIL DISTRICT '), 'district'] = df['office']
df.loc[df.office.str.contains('FOR CITY COUNCIL PARKERSBURG DISTRICT '), 'district'] = df['office']
df.loc[df.office == 'FOR COUNTY COMMISSIONER DIST C', 'district'] = 'C'

In [8]:
def cleanOffice(x):
    if 'DISTRICT' in x:
        index = x.index('DISTRICT')
        x = x[:index+8]
    if 'FOR ' in x:
        index = x.index('FOR ')
        x = x[index+4:]
    if 'COUNTY COMMISSIONER DIST C' in x:
        return 'COUNTY COMMISSIONER DISTRICT'
    if 'WARD' in x:
        index = x.index('WARD')
        x = x[:index]
    elif 'PRESIDENT' in x:
        return 'US PRESIDENT'
    elif 'U.S. SENATOR' in x:
        return 'US SENATE'
    elif 'U.S. HOUSE OF REPRESENTATIVES' in x:
        return 'US HOUSE'
    elif 'STATE SENATOR' in x:
        return 'STATE SENATE'
    elif 'HOUSE OF DELEGATES' in x:
        return 'STATE HOUSE'
    elif 'NON' == x:
        return 'LEWIS COUNTY BOARD OF EDUCATION'  
    elif 'NONPARTISAN BOARD OF EDUCATION' in x:
        return 'BOARD OF EDUCATION'
    else: 
        return x
    return x
    
df['office'] = df['office'].apply(cleanOffice)

In [9]:
def cleanDistrict(x):
    if 'WARD' in x: 
        return x[-6:]
    elif 'DISTRICT' in x: 
        return '00' + x[-1:]
    else: 
        return x
    
df['district'] = df['district'].apply(cleanDistrict)

In [10]:
def getDataverse(x):
    if 'US PRESIDENT' in x:
        return "PRESIDENT"
    elif "US SENATE" in x:
        return "SENATE"
    elif "US HOUSE" in x:
        return "HOUSE"
    elif "STATE" in x or "GOVERNOR" in x or "SECRETARY OF STATE" in x or "AUDITOR" in x or "ATTORNEY GENERAL" in x or "TREASURER" in x or "COMMISSIONER OF AGRICULTURE" in x:
        return "STATE"
    else: 
        return 'LOCAL'
    
df['dataverse'] = df['office'].apply(getDataverse)

In [11]:
df["state"] = 'WEST VIRGINIA'
df["state_po"] = 'WV'
df["state_fips"] = '54'
df["state_cen"] = '55'
df["state_ic"] = '56'

In [12]:
df["stage"] = "GEN"
df["date"] = "2020-11-03"
df["year"] = "2020"

df["mode"] = "TOTAL"

df["special"] = "FALSE"
df["writein"] = "FALSE"
df['magnitude'] = 1

In [13]:
# After county name fix, append on fips codes
fips = pd.read_csv('/Users/jennifermiranda/Documents/GitHub/2020-precincts/help-files/county-fips-codes.csv')

fips['state'] = fips['state'].str.upper()
fips = fips.applymap(str)
df['county_name'] = df['county'].str.upper()
df = df.drop(['county'], axis=1)
df=df.applymap(lambda x: x.strip() if type(x)==str else x)
df = pd.merge(df, fips, on = ['state','county_name'], 
             how = 'left')

In [14]:
df['jurisdiction_name'] = df['county_name']
df['jurisdiction_fips'] = df['county_fips']

In [15]:
df.candidate = df.candidate.str.replace(' +', ' ')
df.candidate = df.candidate.str.strip()
df.candidate = df.candidate.str.replace('ROBERT " BOB" STULTZ', 'ROBERT "BOB" STULTZ')


In [16]:
df['office'] = df['office'].replace({'COUNTY COMMISSIONER DISTRICT': 'COUNTY COMMISSIONER', 
                                           'LEWIS COUNTY BOARD OF EDUCATION': 'BOARD OF EDUCATION',
                                           'COUNCIL-AT-LARGE HUNTINGTON': 'CITY COUNCIL HUNTINGTON',
                                           'COUNCIL-AT-LARGE CITY OF VIENNA': 'CITY COUNCIL VIENNA',
                                           'FAIRMONT DISTRICT': 'CITY COUNCIL FAIRMONT',
                                           'CITY COUNCIL DISTRICT': 'CITY COUNCIL',
                                           'CITY COUNCIL PARKERSBURG DISTRICT': 'CITY COUNCIL PARKERSBURG'})

In [17]:
df.loc[df['candidate'].str.contains('SCOTT MARSHALL CASERTA'), 'office'] = 'MAYOR HUNTINGTON'
df.loc[df['candidate'].str.contains('STEVE WILLIAMS'), 'office'] = 'MAYOR HUNTINGTON'
df.loc[df['candidate'].str.contains('BOB BAILEY'), 'office'] = 'CITY COUNCIL HUNTINGTON'
df.loc[df['candidate'].str.contains('JOSHUA M GARNES'), 'office'] = 'CITY COUNCIL HUNTINGTON'
df.loc[df['candidate'].str.contains('DURON JACKSON'), 'office'] = 'CITY COUNCIL HUNTINGTON'
df.loc[df['candidate'].str.contains('DAVID HARRINGTON'), 'office'] = 'CITY COUNCIL HUNTINGTON'
df.loc[df['candidate'].str.contains('TYLER BOWEN'), 'office'] = 'CITY COUNCIL HUNTINGTON'
df.loc[df['candidate'].str.contains('JOYCE CLARK'), 'office'] = 'CITY COUNCIL HUNTINGTON'

df.loc[df['candidate'].str.contains('TYLER BOWEN'), 'district'] = '001'
df.loc[df['candidate'].str.contains('JOYCE CLARK'), 'district'] = '001'

In [18]:
df.candidate = df.candidate.str.replace("GREG ''HOOTIE '' SMITH", 'GREG "HOOTIE" SMITH')
df.loc[df['candidate'].str.contains('WRITEIN'), 'writein'] = 'TRUE'

df.loc[df['office'].str.contains('US PRESIDENT'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('US SENATE'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('GOVERNOR'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('SECRETARY OF STATE'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('AUDITOR'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('TREASURER'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('COMMISSIONER OF AGRICULTURE'), 'district'] = 'STATEWIDE'
df.loc[df['office'].str.contains('ATTORNEY GENERAL'), 'district'] = 'STATEWIDE'

In [19]:
df.loc[df['candidate'].str.contains('BOB BAILEY'), 'party_detailed'] = 'DEMOCRAT'
df.loc[df['candidate'].str.contains('JOSHUA M GARNES'), 'party_detailed'] = 'REPUBLICAN'
df.loc[df['candidate'].str.contains('DURON JACKSON'), 'party_detailed'] = 'DEMOCRAT'
df.loc[df['candidate'].str.contains('DAVID HARRINGTON'), 'party_detailed'] = 'REPUBLICAN'

# run party simplified again 

In [35]:
df.loc[df['office'].str.contains('FAMILY COURT JUDGE'), 'special'] = 'TRUE'
df.loc[df['office'].str.contains('MAGISTRATE'), 'special'] = 'TRUE'
df.loc[df['office'].str.contains('CIRCUIT CLERK'), 'special'] = 'TRUE'
df.loc[df['office'].str.contains('CIRCUIT CLERK'), 'dataverse'] = 'STATE'

In [38]:
df.loc[df['office'].str.contains('CITY COUNCIL'), 'party_detailed'] = 'NONPARTISAN'
df.loc[df['office'].str.contains('CITY COUNCIL'), 'party_simplified'] = 'NONPARTISAN'
df.loc[df['office'].str.contains('MAYOR'), 'party_detailed'] = 'NONPARTISAN'
df.loc[df['office'].str.contains('MAYOR'), 'party_simplified'] = 'NONPARTISAN'
df.loc[df['office'].str.contains('COURT'), 'party_detailed'] = 'NONPARTISAN'
df.loc[df['office'].str.contains('COURT'), 'party_simplified'] = 'NONPARTISAN'

In [39]:
# Readme Check - No issues 
df["readme_check"] = "FALSE"

# Final step: Remove all trailing white space and put columns in correct order. 
df=df.applymap(lambda x: x.strip() if type(x)==str else x)

df=df[["precinct", "office", "party_detailed", "party_simplified", "mode", "votes", "county_name", "county_fips", "jurisdiction_name",
                      "jurisdiction_fips", "candidate", "district", "dataverse", "year", "stage", "state", "special", "writein", "state_po",
                      "state_fips", "state_cen", "state_ic", "date", "readme_check", "magnitude"]]

df=df.set_index('precinct')

df.to_csv('2020-wv-precinct-general.csv')

In [21]:
df

,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
precinct,,,,,,,,,,,,,,,,,,,,,
PRECINCT 23,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,457,RANDOLPH,54083,RANDOLPH,54083,DONALD J TRUMP,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
PRECINCT 24,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,203,RANDOLPH,54083,RANDOLPH,54083,DONALD J TRUMP,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
PRECINCT 25,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,189,RANDOLPH,54083,RANDOLPH,54083,DONALD J TRUMP,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
PRECINCT 27,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,207,RANDOLPH,54083,RANDOLPH,54083,DONALD J TRUMP,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
PRECINCT 28,US PRESIDENT,REPUBLICAN,REPUBLICAN,TOTAL,241,RANDOLPH,54083,RANDOLPH,54083,DONALD J TRUMP,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRECINCT 137,SURVEYOR,,,TOTAL,5,HARRISON,54033,HARRISON,54033,WRITEIN,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
PRECINCT 143,SURVEYOR,,,TOTAL,46,HARRISON,54033,HARRISON,54033,WRITEIN,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1
PRECINCT 144,SURVEYOR,,,TOTAL,15,HARRISON,54033,HARRISON,54033,WRITEIN,...,WEST VIRGINIA,FALSE,FALSE,WV,54,55,56,2020-11-03,FALSE,1


In [24]:
for x in df['candidate'].unique():
    if 'WRITE' in x:
        print(x)

WRITEIN


In [33]:
x = 'FOR CITY COUNCIL DUNBAR WARD 1'
'00' + x[-1:]

'001'

In [32]:
index = df[df.office == 'CIRCUIT CLERK']
index

,precinct,office,district,candidate,votes,party_detailed,party_simplified,dataverse,state,state_po,...,date,year,mode,special,writein,magnitude,county_name,county_fips,jurisdiction_name,jurisdiction_fips
6850,PRECINCT 2,CIRCUIT CLERK,051,KEITH STOUT,269,DEMOCRAT,DEMOCRAT,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WEBSTER,54101,WEBSTER,54101
6851,PRECINCT 3,CIRCUIT CLERK,051,KEITH STOUT,220,DEMOCRAT,DEMOCRAT,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WEBSTER,54101,WEBSTER,54101
6852,PRECINCT 4,CIRCUIT CLERK,051,KEITH STOUT,202,DEMOCRAT,DEMOCRAT,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WEBSTER,54101,WEBSTER,54101
6853,PRECINCT 5,CIRCUIT CLERK,051,KEITH STOUT,153,DEMOCRAT,DEMOCRAT,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WEBSTER,54101,WEBSTER,54101
6854,PRECINCT 8,CIRCUIT CLERK,051,KEITH STOUT,263,DEMOCRAT,DEMOCRAT,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WEBSTER,54101,WEBSTER,54101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35000,PRECINCT 85,CIRCUIT CLERK,054,CELESTE WALDRON RIDGWAY,468,REPUBLICAN,REPUBLICAN,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WOOD,54107,WOOD,54107
35001,PRECINCT 86,CIRCUIT CLERK,054,CELESTE WALDRON RIDGWAY,552,REPUBLICAN,REPUBLICAN,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WOOD,54107,WOOD,54107
35002,PRECINCT 87,CIRCUIT CLERK,054,CELESTE WALDRON RIDGWAY,379,REPUBLICAN,REPUBLICAN,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WOOD,54107,WOOD,54107
35003,PRECINCT 88,CIRCUIT CLERK,054,CELESTE WALDRON RIDGWAY,503,REPUBLICAN,REPUBLICAN,LOCAL,WEST VIRGINIA,WV,...,2020-11-03,2020,TOTAL,FALSE,FALSE,1,WOOD,54107,WOOD,54107


In [34]:
for x in index['dataverse'].unique():
    print(x)

LOCAL
